In [2]:
pip install ib_insync


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [62]:
from ib_insync import *

# Utilize an instance of IB
ib = IB()

# Connect to the IB server using await in front of the connectAsync method
try:
    await ib.connectAsync('127.0.0.1', 7497, clientId=1)
    print("Connected to Interactive Brokers")
except RuntimeError as e:
    print(f"Runtime Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
# from ib_insync import *

# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=2)

Connected to Interactive Brokers


In [63]:
contract = Future('ES', '202403', 'CME')

In [64]:
await ib.qualifyContractsAsync(contract)

[Future(conId=533620665, symbol='ES', lastTradeDateOrContractMonth='20240315', multiplier='50', exchange='CME', currency='USD', localSymbol='ESH4', tradingClass='ES')]

In [65]:
print(ib.reqMarketDataType(3))

None


In [66]:
from ib_insync import *

# Assume 'ib' is already connected and is an instance of ib_insync.IB()
# Also assume 'contract' has been created and is a valid ib_insync.Contract object

# Use the asynchronous version of the method with await
try:
    historical_data = await ib.reqHistoricalDataAsync(
        contract, endDateTime='', durationStr='30 D',
        barSizeSetting='1 hour', whatToShow='MIDPOINT', useRTH=True
    )
    
    # Proceed with the historical data
    for bar in historical_data:
        print(bar)
except Exception as e:
    print(f"An error occurred: {e}")


BarData(date=datetime.datetime(2024, 1, 26, 8, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Central')), open=4918.5, high=4925.5, low=4914.75, close=4919.5, volume=-1.0, average=-1.0, barCount=-1)
BarData(date=datetime.datetime(2024, 1, 26, 9, 0, tzinfo=zoneinfo.ZoneInfo(key='US/Central')), open=4919.5, high=4933.5, low=4912.75, close=4933.25, volume=-1.0, average=-1.0, barCount=-1)
BarData(date=datetime.datetime(2024, 1, 26, 10, 0, tzinfo=zoneinfo.ZoneInfo(key='US/Central')), open=4933.25, high=4934.25, low=4923.5, close=4930.25, volume=-1.0, average=-1.0, barCount=-1)
BarData(date=datetime.datetime(2024, 1, 26, 11, 0, tzinfo=zoneinfo.ZoneInfo(key='US/Central')), open=4930.25, high=4931.5, low=4909.0, close=4912.5, volume=-1.0, average=-1.0, barCount=-1)
BarData(date=datetime.datetime(2024, 1, 26, 12, 0, tzinfo=zoneinfo.ZoneInfo(key='US/Central')), open=4912.5, high=4917.5, low=4907.5, close=4912.0, volume=-1.0, average=-1.0, barCount=-1)
BarData(date=datetime.datetime(2024, 1, 26, 13, 0, tzi

In [69]:
df = util.df(historical_data)
print(df)

                         date     open     high      low    close  volume  \
0   2024-01-26 08:30:00-06:00  4918.50  4925.50  4914.75  4919.50    -1.0   
1   2024-01-26 09:00:00-06:00  4919.50  4933.50  4912.75  4933.25    -1.0   
2   2024-01-26 10:00:00-06:00  4933.25  4934.25  4923.50  4930.25    -1.0   
3   2024-01-26 11:00:00-06:00  4930.25  4931.50  4909.00  4912.50    -1.0   
4   2024-01-26 12:00:00-06:00  4912.50  4917.50  4907.50  4912.00    -1.0   
..                        ...      ...      ...      ...      ...     ...   
232 2024-03-08 08:30:00-06:00  5165.25  5188.75  5163.25  5188.25    -1.0   
233 2024-03-08 09:00:00-06:00  5188.25  5193.00  5180.00  5186.25    -1.0   
234 2024-03-08 10:00:00-06:00  5186.25  5187.25  5149.75  5150.75    -1.0   
235 2024-03-08 11:00:00-06:00  5150.75  5159.75  5133.50  5136.00    -1.0   
236 2024-03-08 12:00:00-06:00  5136.00  5143.50  5121.75  5124.75    -1.0   

     average  barCount  
0       -1.0        -1  
1       -1.0        -1  


In [74]:
order = Order(action='SELL', totalQuantity=5, orderType='LMT', lmtPrice=5300)

In [75]:
trade = ib.placeOrder(contract, order)

Error 354, reqId 15: You are trying to submit an order without having market data for this instrument. IBKR strongly recommends against this kind of blind trading which may result in erroneous or unexpected trades. Restriction is specified in Precautionary Settings of Global Configuration/Presets.
Canceled order: Trade(contract=Future(conId=533620665, symbol='ES', lastTradeDateOrContractMonth='20240315', multiplier='50', exchange='CME', currency='USD', localSymbol='ESH4', tradingClass='ES'), order=Order(orderId=15, clientId=1, action='SELL', totalQuantity=5, orderType='LMT', lmtPrice=5300), orderStatus=OrderStatus(orderId=15, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 3, 8, 19, 9, 44, 410450, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 3, 8, 19, 9

In [79]:
order = Order(action='BUY', totalQuantity=1, orderType='LMT', lmtPrice=5140)
trade = ib.placeOrder(contract, order)

Error 354, reqId 19: You are trying to submit an order without having market data for this instrument. IBKR strongly recommends against this kind of blind trading which may result in erroneous or unexpected trades. Restriction is specified in Precautionary Settings of Global Configuration/Presets.
Canceled order: Trade(contract=Future(conId=533620665, symbol='ES', lastTradeDateOrContractMonth='20240315', multiplier='50', exchange='CME', currency='USD', localSymbol='ESH4', tradingClass='ES'), order=Order(orderId=19, clientId=1, action='BUY', totalQuantity=1, orderType='LMT', lmtPrice=5140), orderStatus=OrderStatus(orderId=19, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 3, 8, 19, 34, 37, 144366, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 3, 8, 19, 3

In [76]:
from ib_insync import *

# Assuming 'ib' is already connected and is an instance of ib_insync.IB()

# Use the asynchronous version of the accountSummary function with await
account_summary = await ib.accountSummaryAsync()

# Find the total cash balance
balance = next((item for item in account_summary if item.tag == 'TotalCashBalance'), None)
print("Account Balance:", balance.value if balance else "No balance found")


Account Balance: 951328.50


Error 201, reqId 14: Order rejected - reason:The Available Funds in your Commodities segment are insufficient to cover the change in the margin<br>requirements in your Commodities segment should this order execute. In order to obtain the desired position,<br>your Commodities Net Liquidation Value [951092.25 USD] must exceed the new total initial Margin of [987185.37<br>USD]. Review the FAQ:<a href="https://prod.ibkr-web.com/ibconfluence/pages/viewpage.action?pageId=30122600"> Why does the credit check for my futures order say I do not have enough funds in my Net<br>Liquidation Value?</a>
Canceled order: Trade(contract=Future(conId=533620665, symbol='ES', lastTradeDateOrContractMonth='20240315', multiplier='50', exchange='CME', currency='USD', localSymbol='ESH4', tradingClass='ES'), order=Order(orderId=14, clientId=1, permId=1777033502, action='SELL', totalQuantity=100.0, orderType='LMT', lmtPrice=5300.0, auxPrice=0.0), orderStatus=OrderStatus(orderId=14, status='Cancelled', filled=0.0,

In [60]:
import alpaca_trade_api as tradeapi

# Set your API key and secret
API_KEY = 'your_api_key'
API_SECRET = 'your_api_secret'
BASE_URL = 'https://paper-api.alpaca.markets'

# Initialize the API connection
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')

# Get stock data using get_bars instead of get_barset
bars = api.get_bars('AAPL', tradeapi.TimeFrame.Day, limit=10).df
for index, row in bars.iterrows():
    print(index, row['open'], row['high'], row['low'], row['close'], row['volume'])


Task exception was never retrieved
future: <Task finished name='Task-97' coro=<IB.connectAsync() done, defined at /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ib_insync/ib.py:1739> exception=TimeoutError()>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/asyncio/tasks.py", line 690, in _wrap_awaitable
    return (yield from awaitable.__await__())
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/asyncio/tasks.py", line 492, in wait_for
    fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ib_insync/ib.py", line 1748, in connectAsync
    await self.clie

HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/AAPL/bars?timeframe=1Day&adjustment=raw&limit=10

In [58]:
pip install alpaca-trade-api

  Obtaining dependency information for alpaca-trade-api from https://files.pythonhosted.org/packages/0b/b2/4557d0a4c837b020bc5c8971e8fde8b976e332d5c225476699e0b5e30b41/alpaca_trade_api-3.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for websockets<11,>=9.0 from https://files.pythonhosted.org/packages/47/4d/f2e28f112302d3bc794b74ae64656255161d8223f4d47bd17d40cbb3629e/websockets-10.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for msgpack==1.0.3 from https://files.pythonhosted.org/packages/3a/57/c90cb0b1ab68650ff0068240449bc17c269069bd898cf283dc8db72e8788/msgpack-1.0.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for aiohttp<4,>=3.8.3 from https://files.pythonhosted.org/packages/20/1c/b7e668a1583cbb1e5060b45baee68a50123f66470ac463f81f7e3b3c3bab/aiohttp-3.9.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for PyYAML==6.0.1 from https://files.pythonhosted.org/packages/57/c5/5d0


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
order = Order(action='BUY', totalQuantity=100, orderType='LMT', lmtPrice=5200)
trade = ib.placeOrder(contract, order)

ConnectionError: Not connected

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [9]:
import alpaca_trade_api as tradeapi

# Set your API key and secret
API_KEY = 'your_api_key'
API_SECRET = 'your_api_secret'
BASE_URL = 'https://paper-api.alpaca.markets'  # or 'https://api.alpaca.markets' for live trading

# Initialize the API connection
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')

# Get stock data
barset = api.get_bars('AAPL', 'day', limit=10)
aapl_bars = barset['AAPL']
for bar in aapl_bars:
    print(bar.t, bar.o, bar.h, bar.l, bar.c, bar.v)


HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/AAPL/bars?timeframe=day&adjustment=raw&limit=10

In [8]:
pip install alpha_vantage

  Obtaining dependency information for alpha_vantage from https://files.pythonhosted.org/packages/ba/b4/d95f9e0eccea6732bab5a079772d453a4f0b68a9f63740d9cf320f92beaa/alpha_vantage-2.3.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from alpha_vantage.timeseries import TimeSeries

# Set your API key
API_KEY = 'your_api_key'

# Initialize the API connection
ts = TimeSeries(key=API_KEY, output_format='pandas')

# Get stock data
data, metadata = ts.get_daily(symbol='AAPL', outputsize='compact')
print(data)

            1. open   2. high   3. low  4. close    5. volume
date                                                         
2024-03-19   174.34  176.6050  173.030    176.08   55215244.0
2024-03-18   175.57  177.7100  173.520    173.72   75604184.0
2024-03-15   171.17  172.6200  170.285    172.62  121752699.0
2024-03-14   172.91  174.3078  172.050    173.00   72571635.0
2024-03-13   172.77  173.1850  170.760    171.13   51948951.0
...             ...       ...      ...       ...          ...
2023-10-31   169.35  170.9000  167.900    170.77   44846017.0
2023-10-30   169.02  171.1700  168.870    170.29   51130955.0
2023-10-27   166.91  168.9600  166.830    168.22   58499129.0
2023-10-26   170.37  171.3775  165.670    166.89   70625258.0
2023-10-25   171.88  173.0600  170.650    171.10   57156962.0

[100 rows x 5 columns]


In [15]:
import requests
from bs4 import BeautifulSoup

# Set the URL for MarketAux
url = 'https://www.marketaux.com/'

try:
    # Send a request to the website and parse the HTML content
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract financial headlines
    headlines = soup.find_all('h2', class_='entry-title')
    for headline in headlines:
        print(headline.text)
        
except Exception as e:
    print("An error occurred:", e)

In [15]:
import requests
from dateutil.parser import parse

def get_headlines():
    url = "https://api.marketaux.com/v1/news/all?symbols=^GSPC&filter_entities=true&language=en&api_token="
    
    cleaned_data = []
    for _ in range(3):
        response = requests.get(url)
        news_headlines = response.json()
        print(news_headlines)  # Print the response content
        for article in news_headlines['data']:
            date = article['published_at']
            title = article['title']
            cleaned_data.append({'date': date, 'headline': title})
    
    return cleaned_data

def extract_headlines(data):
    # Initialize an empty dictionary to hold the headlines
    headlines = {}
    
    # Loop through each article in the data file
    for article in data['articles']:
        # Extract the date from the 'publishedAt' field and convert it to a datetime object
        date = parse(article['publishedAt']).date()
        
        # Convert the date to a string in the format 'YYYY-MM-DD'
        date_str = date.strftime('%Y-%m-%d')
        
        # If the date is not already in the headlines dictionary, add it with an empty list as the value
        if date_str not in headlines:
            headlines[date_str] = []
        
        # Append the article's title to the list of headlines for this date
        headlines[date_str].append(article['title'])
data = get_headlines()
extract_headlines(data)

{'error': {'code': 'invalid_api_token', 'message': 'An invalid API token was supplied.'}}


KeyError: 'data'